In this notebook, we will calculate features using Trotterization with a noise-free classical simulator (AerSimulator).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']
/home/jovyan/fourier_learning_ibm


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [5]:
backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend(
    qpu_name="ibm_marrakesh"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend noiseless simulator: AerSimulator('aer_simulator')

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['measure', 'reset', 'id', 'x', 'sx', 'cz']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

### Prepare parameters and load dataset

In [6]:
n_qubits = 4
# C = 3
# C = 3 * (n_qubits - 1)
C = 5
threshold = 0.6

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
    orient="records",
)
n_samples = 55

# Note that K = n_features-1.
# n_features counts only the number of real features, also the one of t=0, i.e., constant term.
# The number of all features, including imaginary numbers, is 2K+1.
n_features = len(df_n_step)

times = df_n_step["t"].values

display(df_n_step)
print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_features: {n_features}")
print(f"times: {times}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

,t,n_step,sta_fidelity
0,0.000000,1,1.000000
1,0.628319,1,0.960905
2,1.256637,1,0.934116
3,1.884956,2,0.923847
4,2.513274,2,0.899309
5,3.141593,3,0.878686
6,3.769911,3,0.862589
7,4.398230,4,0.840110
8,5.026548,5,0.813450
9,5.654867,5,0.800024


n_qubits: 4
C: 5
threshold: 0.6
n_samples: 55
n_features: 19
times: [ 0.          0.62831853  1.25663706  1.88495559  2.51327412  3.14159265
  3.76991118  4.39822972  5.02654825  5.65486678  6.28318531  6.91150384
  7.53982237  8.1681409   8.79645943  9.42477796 10.05309649 10.68141502
 11.30973355]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.3949339961, -0.0881126084, 0.5169533955]",5.547457,5.547450,6.426300e-06
1,1,"[0.1829492443, -0.3761751478, 0.4408756079]",3.841905,3.841881,2.462770e-05
2,2,"[0.2841284266, 0.3112471996, -0.4046243738]",1.129636,1.129639,3.412100e-06
3,3,"[-0.0819280326, -0.2133520868, 0.7047198806]",4.731457,4.731416,4.172410e-05
4,4,"[0.2749649723, 0.616884329, -0.1081506987]",0.852425,0.852410,1.459610e-05
5,5,"[-0.3572377034, 0.0714901473, -0.5712721494]",0.828226,0.828227,1.070800e-06
6,6,"[0.4567030859, 0.1835342376, 0.3597626765]",3.697788,3.697784,4.380700e-06
7,7,"[-0.1441345638, 0.4663640207, 0.3895014155]",1.258985,1.258995,1.064270e-05
8,8,"[0.45117141930000004, -0.49489387500000004, -0...",3.279570,3.279566,4.387400e-06
9,9,"[-0.4631642178, -0.3509909223, 0.1858448599000...",1.577586,1.577586,7.130000e-08


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [7]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": df_n_step["n_step"].values,
    "backend": backend_sim_noiseless,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [8]:
_, jobs, _ = run_job(config)

Preparing circuits for sample 0/55
Preparing circuits for sample 1/55
Preparing circuits for sample 2/55
Preparing circuits for sample 3/55
Preparing circuits for sample 4/55
Preparing circuits for sample 5/55
Preparing circuits for sample 6/55
Preparing circuits for sample 7/55
Preparing circuits for sample 8/55
Preparing circuits for sample 9/55
Preparing circuits for sample 10/55
Preparing circuits for sample 11/55
Preparing circuits for sample 12/55
Preparing circuits for sample 13/55
Preparing circuits for sample 14/55
Preparing circuits for sample 15/55
Preparing circuits for sample 16/55
Preparing circuits for sample 17/55
Preparing circuits for sample 18/55
Preparing circuits for sample 19/55
Preparing circuits for sample 20/55
Preparing circuits for sample 21/55
Preparing circuits for sample 22/55
Preparing circuits for sample 23/55
Preparing circuits for sample 24/55
Preparing circuits for sample 25/55
Preparing circuits for sample 26/55
Preparing circuits for sample 27/55
Pr

In [9]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_14 Im,f_15 Re,f_15 Im,f_16 Re,f_16 Im,f_17 Re,f_17 Im,f_18 Re,f_18 Im,expected_value
0,0,1.0,0.559947,0.407132,0.036789,0.127444,-0.098054,-0.033022,-0.303064,0.164285,...,-0.690341,0.339422,-0.305664,0.174201,-0.055758,0.249396,-0.080039,0.468196,0.229590,5.547457
1,1,1.0,0.692743,0.509144,0.137678,0.386567,-0.301019,0.103584,-0.265812,-0.234759,...,-0.559732,0.292897,-0.469406,0.345415,0.081826,-0.158607,0.440517,-0.746004,0.394469,3.841905
2,2,1.0,0.758867,-0.240900,0.312844,-0.171965,-0.041572,-0.249366,-0.143930,-0.319206,...,-0.029909,0.073431,0.079258,-0.185783,0.204221,-0.146012,0.096527,0.105064,-0.168316,1.129636
3,3,1.0,0.543812,0.286092,-0.077930,-0.173575,-0.296312,-0.706375,-0.063154,-0.737831,...,0.189904,0.000597,0.386946,0.135512,0.285520,0.273575,0.237291,0.267123,0.160038,4.731457
4,4,1.0,0.876439,-0.303968,0.621926,-0.373682,0.398266,-0.596260,0.267088,-0.527982,...,-0.359158,-0.109983,-0.228432,-0.078802,-0.145565,-0.044410,-0.020807,-0.062182,-0.086400,0.852425
5,5,1.0,0.547740,-0.389446,0.015685,-0.088297,-0.079811,0.123514,-0.207041,-0.078080,...,0.667988,0.295294,0.004951,-0.207291,-0.061401,-0.151579,0.103868,0.046993,-0.034697,0.828226
6,6,1.0,0.674179,0.316522,0.200297,0.186245,0.135157,-0.026459,0.195120,0.243159,...,-0.352270,-0.092773,-0.529297,0.479310,-0.267363,0.560239,0.288822,0.354053,0.429175,3.697788
7,7,1.0,0.844791,-0.108400,0.520776,-0.147985,0.245230,-0.341895,0.082656,-0.358387,...,0.201309,0.263735,0.118663,0.431500,-0.059605,0.586797,-0.191943,0.499332,-0.223326,1.258985
8,8,1.0,0.706850,0.438702,0.163075,0.393035,-0.286849,0.206849,-0.251729,-0.096278,...,0.025448,0.347119,0.373982,-0.045350,0.408489,-0.133295,0.221235,0.052203,-0.076568,3.279570
9,9,1.0,0.806190,0.010056,0.332599,0.058008,0.037105,0.262187,-0.036029,0.339556,...,-0.506007,0.393067,-0.211268,0.743825,-0.007482,0.872421,0.028808,0.675195,-0.020317,1.577586
